In [1]:
import healpy as hp
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
dist=0.6462359932748121

In [3]:
def gen_centers_fast(n, all_pixels, nside1 = 2**10, nside2=2**17):
    
    def n_pix2ra_dec(npix, nside):
        theta, phi = hp.pix2ang(nside=nside, ipix=npix, nest=True, lonlat=True)
        sc = SkyCoord(l=theta*u.degree, b=phi*u.degree, frame='galactic')
        return sc.icrs.ra.degree, sc.icrs.dec.degree

    
    ans = np.random.choice(all_pixels, size=n)
    
    ras, decs = n_pix2ra_dec(ans, nside1)
    
    if nside1 != nside2:
        l, b = hp.pix2ang(nside=nside1, ipix=ans, nest=True, lonlat=True)
        ans = hp.ang2pix(nside=nside2, nest=True, lonlat=True, theta=l, phi=b)
    
    return pd.DataFrame({'pix':ans, 'ra':ras, 'dec':decs})

In [4]:
planck = pd.read_csv('../src_data/planck_z.csv', index_col='index')
planck.head()

,Name,RAdeg,DEdeg,z,MSZ,MCXC,RedMAPPer,pix,theta,phi
index,,,,,,,,,,
0,PSZ2 G000.04+45.13,229.190512,-1.017222,0.1198,3.962411,J1516.5-0056,RMJ151653.9-010506.3,2,0.000708,0.787757
1,PSZ2 G000.13+78.04,203.558683,20.255990,0.1710,5.122391,J1334.1+2013,RMJ133408.7+201453.0,3,0.002410,1.362092
2,PSZ2 G000.40-41.86,316.084485,-41.354169,0.1651,5.297053,J2104.3-4120,,34,0.007034,-0.730609
3,PSZ2 G000.77-35.69,307.972844,-40.598725,0.3416,6.333562,J2031.8-4037,,16,0.013527,-0.623072
4,PSZ2 G002.08-68.28,349.632431,-36.332559,0.1400,2.835297,,,32,0.036425,-1.191738


In [5]:
planck5=planck[planck['pix']==5]
planck5.shape

(36, 10)

In [14]:
a = gen_centers_fast(planck5, dist, 10)

TypeError: only size-1 arrays can be converted to Python scalars

In [6]:
def gen_patches_pixels(cat, radius, nside=2**10):
    
    all_pixels = np.array([])
    for i in range(cat.shape[0]):
        ra = cat['RAdeg'].iloc[i]
        dec = cat['DEdeg'].iloc[i]
        sc = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
        vec = hp.ang2vec(theta=sc.galactic.l.degree, 
                         phi=sc.galactic.b.degree, 
                         lonlat=True)
        pixels = hp.query_disc(nside=nside, vec=vec, radius=np.radians(radius), nest=True).flatten()
        all_pixels = np.concatenate([all_pixels, pixels], axis=0)
    
    all_pixels = np.unique(all_pixels)
    return all_pixels

In [59]:
all_centers_in5_nside10 = gen_patches_pixels(planck5, dist)

In [60]:
all_centers_in5_nside10.shape

(13853,)

In [61]:
np.save('/home/rt2122/data/all_centers_in5_nside10', all_centers_in5_nside10)

In [8]:
all_centers_in5_nside10 = np.load('/home/rt2122/data/all_centers_in5_nside10.npy')

In [11]:
cnt = gen_centers_fast(20, all_centers_in5_nside10, 2**10, 2**17)

In [10]:
all_centers_in5_nside10 = all_centers_in5_nside10.astype(np.int64)

In [12]:
cnt.head()

,pix,ra,dec
0,22951680682,132.221748,48.009135
1,21482986154,100.095108,57.330533
2,25322199722,156.208598,49.593828
3,24804468053,148.255913,52.175710
4,21488409258,99.916181,58.030799


In [2]:
from all_p import ClusterFile

In [14]:
cnt.index.name='index'

In [3]:
cf = ClusterFile('_')

In [4]:
cf.params['typ'] = 'cen'
cf.params['id_list'] = 1
cf.params['num'] = 20
cf.params['inpix'] = 5
cf.params['size'] = dist
cf.params['nside'] = 17

In [5]:
cf.file('.csv')

'tcen_il1_n20_in5_len0.6462_hns17.csv'

In [20]:
cnt.to_csv('tmp.csv')

In [6]:
cnt = pd.read_csv('tmp.csv', index_col='index')
cnt.head()

,pix,ra,dec
index,,,
0,22951680682,132.221748,48.009135
1,21482986154,100.095108,57.330533
2,25322199722,156.208598,49.593828
3,24804468053,148.255913,52.175710
4,21488409258,99.916181,58.030799


In [7]:
from os.path import join
cnt.to_csv(join('/home/rt2122/data/train/centers/', cf.file('.csv')))